In [ ]:
!pip install numba
!pip install scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from numba import njit
from scipy.optimize import curve_fit
from scipy.optimize import leastsq

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
@njit
def init_pos(N, L):
    pos = np.random.uniform(0.0,L/2,size=(N,3))
    ph = np.zeros(pos.shape) #placeholder
    return pos - L*np.round(pos/L, 0, ph)

In [ ]:
# Constants
epsilon = 1
sigma = 1
r_c = 2.5*sigma

# Parameters for Line Search and Conjugate Gradient
LStol = 10**(-8)
MaxLSSteps = 100000
ECtol = 10**(-10)
MaxCGSteps = 100000
delta = 0.1

In [ ]:
@njit
def calc_energy(pos):
    u_ = 0
    N = len(pos)
    alpha = 0.0001 * (N ** (-2 / 3))
    #non_conv_term = 4*epsilon*((r_c/sigma)**(-12)-(r_c/sigma)**(-6))
    for i in range(N):
        u_ += alpha * np.dot(pos[i], pos[i])
        for j in range (i+1, N):
            r_ij = pos[j] - pos[i]
            r_ij_norm = np.linalg.norm(r_ij)
            if r_ij_norm <= r_c:
                u_ += 4*epsilon*((r_ij_norm/sigma)**(-12)-(r_ij_norm/sigma)**(-6))
    return u_

In [ ]:
@njit
def calc_force(pos):
    N = len(pos)
    alpha = 0.0001*(N**(-2/3))
    f = np.zeros(pos.shape)
    for i in range(len(pos)):
        f[i] -= 2 * alpha * pos[i]
        for j in range (i+1, N):
            r_ij = pos[j] - pos[i]
            r_ij_norm = np.linalg.norm(r_ij)
            if r_ij_norm <= r_c:
                f[i]+=4*epsilon*(-12*(sigma)*(r_ij_norm)**(-13)+6*(sigma)*(r_ij_norm)**(-7))*r_ij/r_ij_norm
                f[j]-=4*epsilon*(-12*(sigma)*(r_ij_norm)**(-13)+6*(sigma)*(r_ij_norm)**(-7))*r_ij/r_ij_norm
    return f

In [ ]:
@njit
def deriv_dot(pos,force): #derivative and dot product
    d = force/np.sqrt((force**2).sum())
    pos_d = -np.dot((calc_force(pos)).reshape(-1), d.reshape(-1))  
    return pos_d
@njit
def bisection(pos_0,pos_2,u_0,u_2):
    while True:
        der_pos_0 = deriv_dot(pos_0,calc_force(pos_0))
        der_pos_2 = deriv_dot(pos_2,calc_force(pos_2))
        pos_mid = (pos_0 + pos_2) / 2
        der_pos_mid = deriv_dot(pos_mid,calc_force(pos_mid))   
    
        pos_1 = (pos_0+pos_2)/2 #center position
        u_1 = calc_energy(pos_1) #potential energy at center position
        
        if abs(u_1 - u_0) < LStol*abs(u_1) : 
            Varray = [0, 0.5, 1]
            u_arr = np.array([u_0, u_1, u_2])
            pos_x = parabola_min(Varray, u_arr)
            pos_min = pos_0 + pos_x * (pos_2 - pos_0)
                
            return pos_min, calc_energy(pos_min)
            
        if der_pos_mid * der_pos_2 > 0 : 
            pos_2 = pos_mid
        else:
            pos_0 = pos_mid
        u_0, u_2 = calc_energy(pos_0), calc_energy(pos_2)
@njit
def parabola_min(x, y):
   
    V = np.vander(x) # Vandermonde matrix from the positions  
    a, b, c = np.linalg.solve(V, y) # get coefficients of the polynomial
    if a == 0:
        xp = - c / b     # x= -c/b  if a=0, (else divide by 0 encounters)
    else:
        xp = - b / (2 * a) # x = -b/2a 
    return xp
@njit
def ls(pos, f, delta, LStol, MaxLSSteps):
    #line search finds the interval between which we have our minima
    #and bisection method locates the minima

    #clip and normalize
    f = np.clip(f, -10000, 10000)
    f = f / np.sqrt(np.sum(f * f))

    i=0
    j=0
    pos_0 = pos
    u_0 = calc_energy(pos_0)
    #we could set delta=0.5 until Δu=10^-2 and after that to 0.01. it can reduce time.
    delta=0.01
    
    pos_1 = pos_0 + delta*f
    u_1 = calc_energy(pos_1)
    pos_2 = pos_1 + delta*f
    u_2 = calc_energy(pos_2)    
    
    while i<MaxLSSteps:
        if u_0>u_1 and u_1<u_2:
            return bisection(pos_0,pos_2,u_0,u_2)
        elif u_0<u_1 and i==0:
            #for case it skips the minima in first step
            return bisection(pos_0,pos_1,u_0,u_1)
        else:
            pos_0, pos_1 = pos_1, pos_2
            u_0, u_1 = u_1, u_2
            pos_2 = pos_1 + delta*f
            u_2=calc_energy(pos_2)
        if abs((u_1 - u_2)/u_2) < 10**-2:
            delta=0.01
        i+=1 
    return pos_1,u_2

In [ ]:
@njit
def cg(pos):
    #PES is an array to save all Potential Energy values
    PES = []
    
    #Initial values
    forces = calc_force(pos)
    PE = calc_energy(pos)
    dir = forces

    #Big enough to enter the loop
    old_PE = 10000000

    #Iterator of CG
    CGStep = 0

    #Conjugate Gradient
    while abs(PE - old_PE) > ECtol*abs(PE) and CGStep < MaxCGSteps:

        old_PE = PE
        CGStep += 1
        PE = ls(pos, dir, delta, LStol, MaxLSSteps)[1]
        pos = ls(pos, dir, delta, LStol, MaxLSSteps)[0]

        PES.append(PE)

        old_forces = forces
        forces = calc_force(pos)

        gamma = np.sum((forces - old_forces) * forces) / np.sum(old_forces * old_forces)
        dir = forces + gamma * dir
    return CGStep, PE, PES, pos

In [ ]:
#Help function for plotting and Task 4
def annot_max(x,y, ax=None):
    xmin = x[np.argmin(y)]
    ymin = y.min()
    text= "x={:.3f}, y={:.3f}".format(xmin, ymin)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=1", fc="w", ec="k", lw=0.72)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(xmin, ymin), xytext=(0.94,0.96), **kw)

def Task4(n_array,u_min):
    def func(params, n_array, u_min):
        a, b, c = params[0], params[1], params[2]
        residual = u_min - (a*n_array**(2/3)+b*n_array+c)
        return residual
    u_macro=np.zeros(len(u_min))
    params = [0, 0, 0]
    result = leastsq(func, params, (n_array, u_min))
    a, b, c = result[0][0], result[0][1], result[0][2]
    for i in range(len(u_min)):
        u_macro[i] = a*n_array[i]**(2/3)+b*n_array[i]+c
    
    plt.figure(figsize = (18, 12))
    plt.grid()
    plt.plot(n_array,u_min-u_macro)
    plt.xlabel('N')
    plt.ylabel('U_min-U_macro')
    plt.title('task 4')
    plt.show()

    print("The parameters of magic clusters")
    print(f"a = {c}")
    print(f"b = {a}")
    print(f"c = {b}")

In [ ]:
def run(N, M):
    res = []
    n_array = np.arange(2, N+1, 1)
    cgsteps = []
    umin_n = []
    for i in range(2, N+1):
        L = (i / 0.01) ** (1 / 3)
        for k in range(M):
            t = init_pos(i, L)
            res.append(list(cg(t)))
        
        #Capture needed values and index of minimal energy
        new = np.array(res)
        energy_array = new[:, 1]
        index = np.argmin(energy_array)

        #Task 2
        umin_n.append(new[index, 1])
        cgsteps.append(new[index, 0])

        #Task 3
        U_min = np.array(new[index, 2])
        Steps = np.arange(1, len(U_min)+1)

        #Plot Task 3
        plt.figure(figsize = (18, 12))
        plt.grid()
        plt.plot(Steps, U_min, 'y')
        plt.xlabel('CGStep')
        plt.ylabel('U_min')
        plt.title('Task 3 - N = %i' %i)
        annot_max(Steps,U_min)
        plt.show()
        
    #Plot Task 2a
    plt.figure(figsize = (18, 12))
    plt.grid()
    plt.plot(n_array, cgsteps, 'r')
    plt.xlabel('N')
    plt.ylabel('CG Steps')
    plt.title('Task 3')
    plt.show()

    #Plot Task 2b
    plt.figure(figsize = (18, 12))
    plt.grid()
    plt.plot(n_array, umin_n, 'b')
    plt.xlabel('N')
    plt.ylabel('U_min')
    plt.title('Task 3')
    plt.show()

    #Plot Task 4
    Task4(n_array, umin_n)

run(25,10)
